In [1]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.common.action_chains import ActionChains
import winsound

In [ ]:
#URL parsing
#Check URL format:
#https://www.pets4homes.co.uk/sale/reptiles/local/
#https://www.pets4homes.co.uk/sale/reptiles/local/birmingham/
#https://www.pets4homes.co.uk/sale/reptiles/local/birmingham/page-11/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/local/page-35/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/durham/page-2/
#https://www.pets4homes.co.uk/sale/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/adoption/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/stud/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/sale/dogs/local/aberdeen/
#URL format is kind of pain to work with, probably should scrape by type of listing, type of pet and city separately.
#species or breed would be listed in the dataframe, no need to be scraped individually.
#so the format would be:
#https://www.pets4homes.co.uk/<type of listing>/<type of pets>/local/<city>/<page>
#page don't exist in URL for page 1.

In [ ]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
listing_type = ['sale','adoption','stud']
pet_type = ['dogs','cats','reptiles','rodents','rabbits','horses','invertebrates','livestock', 'poultry', 'birds','fish']
cities = ['aberdeen','ayr_st-ives', 'barnsley_wimborne','basingstoke','bath','bedfordshire','birmingham','blackpool_plymouth',
         'bolton_appleby-in-westmorland', 'bournemouth', 'bradford_holsworthy', 'bridgend_brechin', 'brighton',
          'bristol_south-west-england', 'cambridge_gloucester', 'cardiff-county', 'carlisle', 'chelmsford', 'cheltenham',
         'chester', 'chesterfield', 'colchester', 'cornwall', 'coventry', 'crawley_witney', 'crewe', 'darlington', 'derby', 
         'devon', 'doncaster', 'dorset', 'dudley_cramlington', 'dundee', 'durham', 'eastbourne', 'edinburgh', 'essex', 'exeter',
         'glasgow', 'gloucester', 'grimsby', 'guildford', 'hampshire', 'hereford', 'hertfordshire', 'huddersfield']
browser = webdriver.Chrome(ChromeDriverManager().install())
page = 1

In [ ]:
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            url = url_template.format(listing, pets, city)
            options = webdriver.ChromeOptions()
            browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)
            browser.get(url)
            title = (
                WebDriverWait(driver=browser, timeout=10)
                .until(visibility_of_element_located((By.CSS_SELECTOR, "h1"))).text
            )
            content = browser.page_source
            soup = BeautifulSoup(content, 'lxml')
            print(url)
            sleep(random.randint(10,15))
            
            sleep(random.randint(10,30))

In [2]:
def check_captcha (soup):
    if 'I am human' in soup.get_text():
            print('captcha detected')
            duration = 5000  # milliseconds
            freq = 440  # Hz
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
    else:
        print('no captcha detected, proceed to scrape')

In [ ]:
def check_if_empty (soup):
    if 'We found 0' in soup.get_text():
            print('no result found')
            

In [3]:
def scrape_header(soup):
    df = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
    for each in soup.find_all("script", type="application/ld+json"):
        entries = each.get_text()
        split = entries.split("\n")
        Title = split[3].replace('"name": "', '').replace('",', '')
        Description = split[4].replace('"description": "', '').replace('",', '').replace('\n,', '')
        Photo = split[5].replace('"image": "', '').replace('",', '')
        Category = split[7].replace('"category": "', '').replace('",', '')
        Price = split[11].replace('"price": "', '').replace('",', '')
        url =split[13].replace('"url": "', '').replace('",', '')
        seller_type = split[15].replace('"type": "', '').replace('",', '')
        seller_name = split[16].replace('"name":  "', '').replace('"', '')
        df = df.append({"Title":Title,"Description":Description,"photo_link":Photo,"category":Category, 
                                      "price":Price,"url":url, "seller_type":seller_type, "seller_name":seller_name,
                                      "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df

In [13]:
def scrape_content(soup):
    df1 = pd.DataFrame(columns=["Title","price","species", "age", "gender", "description",
                                "seller_name", "seller_location", "seller_type", "listing_type","pet_type"])
    for each in soup.find_all("div", class_="mn aj"):
        entries = each.get_text()
        title = each.find('h2', {'class':'wn'}).text
        price = each.find('span', {'class':'xn'}).text
        info = each.find_all('span', {'class':'rv'})
        species = info[0].text
        try:
            age = info[1].text.replace('Age:  ', '')
        except:
            age = 'unknown'
        try:
            gender = info[2].text
        except:
            gender = 'unknown'
        description = each.find('span', {'class':'yn'}).text
        seller_name = each.find('span', {'class':'En'}).text
        seller_location = each.find('span', {'class':'Ct'}).text
        seller_type = each.find('div', {'class':'se sm te'}).text
        df1 = df1.append({"Title":title, "price":price, "species":species, "age":age, "gender":gender,
                              "description":description, "seller_name":seller_name, "seller_location": seller_location,
                              "seller_type":seller_type, "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df1

# For testing page loops

In [14]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
#https://www.pets4homes.co.uk/sale/dogs/local/huddersfield/page-2/
url_template_pages = "https://www.pets4homes.co.uk/{}/{}/local/{}/page-{}/"
listing_type = ['sale']
pet_type = ['dogs']
cities = ['aberdeen']

In [16]:
page = 1
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            while page > 0:
                file = 'data_by_cities/header_'+str(city)+'.csv'
                file1 = 'data_by_cities/body_'+str(city)+'.csv'
                try: 
                    df_header = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
                    print('file successfully opened')
                except:
                    print('file not found, creating new file')
                    df_header = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
                    df_header.to_csv(file, sep='\t', encoding='utf-8')
                try: 
                    df_body = pd.read_csv(file1 , sep='\t', encoding='utf-8', index_col=0)
                    print('file successfully opened')
                except:
                    print('file not found, creating new file')
                    df_body = pd.DataFrame(columns=["Title","price","species", "age", "gender", "description",
                                                    "seller_name", "seller_location", "seller_type" , "listing_type","pet_type"])
                    df_body.to_csv(file1, sep='\t', encoding='utf-8')
                headersize = df_header.shape
                bodysize = df_body.shape
                if page == 1:
                    url = url_template.format(listing, pets, city)
                else:
                    url = url_template_pages.format(listing, pets, city, page)
                options = webdriver.ChromeOptions()
                browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)
                browser.get(url)
                title = (
                    WebDriverWait(driver=browser, timeout=10)
                    .until(visibility_of_element_located((By.CSS_SELECTOR, "h1"))).text
                )
                content = browser.page_source
                print(city, page)
                soup = BeautifulSoup(content, 'lxml')
                check_captcha(soup)
                if 'We found 0' in soup.get_text():
                    print('no result found')
                    break
                df_header = df_header.append(scrape_header(soup))
                df_body = df_body.append(scrape_content(soup))
                #drop duplicates
                df_header = df_header[~df_header.duplicated()].reset_index(drop=True)
                df_header.to_csv(file, sep='\t', encoding='utf-8')
                df_body = df_body[~df_body.duplicated()].reset_index(drop=True)
                df_body.to_csv(file1, sep='\t', encoding='utf-8')
                if df_body.shape == bodysize:
                    if df_header.shape == headersize:
                        print('no new data')
                        break
                    else:
                        winsound.Beep(600, 1000)
                        input('html tags changed, please kill program')
                sleep(random.randint(10,15))
                print('going to next page')
                clear_output(wait=True)
                page +=1
            sleep(random.randint(10,15))
            print('going to next listing type')
        sleep(random.randint(10,15))
        print('going to next pet type')
    sleep(random.randint(15,20))
    print('going to next city')

file not found, creating new file
file not found, creating new file



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\silve\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\silve\AppData\Local\Temp/ipykernel_10908/2226946896.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)


aberdeen 1
no captcha detected, proceed to scrape
file successfully opened
file successfully opened



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\silve\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


aberdeen 2
no captcha detected, proceed to scrape
file successfully opened
file successfully opened



...

aberdeen 3
no captcha detected, proceed to scrape
file successfully opened
file successfully opened



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\silve\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


aberdeen 4
no captcha detected, proceed to scrape
file successfully opened
file successfully opened



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\silve\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


aberdeen 5
no captcha detected, proceed to scrape
no new data


In [17]:
df_body.head(30)

,Title,price,species,age,gender,description,seller_name,seller_location,seller_type,listing_type,pet_type
0,KC Pedigree Chihuahuas - Russian lines,"£2,750",Chihuahua,5 weeks,3 male / 3 female,These unique and beautiful full KC registered ...,Kentwone Klub Chihuahuas,Aberdeen,Breeder,sale,dogs
1,Last 3 remaining pups!,"£2,750",English Bulldog,12 weeks,6 female,"Here we have 5 lilac Merle girls for sale, som...",Ewen R.,Banchory,Individual,sale,dogs
2,Family Cudle Companion Dog,"£3,500",Japanese Shiba Inu,7 weeks,1 male,This handsome boy was born on 12th April 2022....,Jelena P,Aberdeen,Breeder,sale,dogs
3,German Shepherd KC puppies/Low Inbred,"£1,500",German Shepherd,3 weeks,4 male / 4 female,"Our beautiful Kennel club registered, long hai...",Michelle B.,Laurencekirk,Individual,sale,dogs
4,rottweiler puppies,"£1,400",Rottweiler,7 weeks,1 male / 3 female,"rottweiler puppy's for sale all microchipped, ...",Gillian M.,Stonehaven,Individual,sale,dogs
5,Miniature poodles,"£2,500",Miniature Poodle,4 weeks,3 male,"Three male miniature poodles for sale, they wi...",Jie B.,Westhill,Individual,sale,dogs
6,Labrador puppies looking for forever homes,£950,Labrador Retriever,4 weeks,5 male / 4 female,Update*** 2 x golden boys sold. one golden gir...,Emily G.,Banchory,Individual,sale,dogs
7,Gorgeous cocker spaniel pups - show type,"£1,750",Cocker Spaniel,8 weeks,3 male / 1 female,We are pleased to announce that our gorgeous ...,Kerrie M.,Aberdeen,Individual,sale,dogs
8,Beautiful Black Labrador Puppies,£850,Labrador Retriever,9 weeks,2 male,Beautiful black Labrador puppies looking for l...,JOHN M.,Banchory,Individual,sale,dogs
9,Beautiful Blue Toy Poodle Puppies. Ready now.,"£2,500",Toy Poodle,14 weeks,4 male / 1 female,Rare blue toy poodle puppies. Genetically hea...,Diane T.,Peterhead,Individual,sale,dogs


It works, let's make new notebook for tidier code